Packages import 

working version : https://www.kaggle.com/code/daniil19189/exploration-of-data

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path

data_train_path = '/kaggle/input/hms-harmful-brain-activity-classification/train.csv'

Data import

In [ ]:
# Load the training data from a CSV file into a DataFrame
data = pd.read_csv(data_train_path)

# Create a copy of the original DataFrame to avoid reloading data multiple times
df = data.copy()  # Backup to avoid reloading

print(df.head())

Based on the description of the datasets, the highest possible data quality is the agreement of five experts

In [ ]:
# Group the data by 'eeg_id' and count the unique 'expert_consensus' values
# This metric is used to assess the quality of the EEG recordings

counts = df.groupby('eeg_id')['expert_consensus'].nunique() # используем как метрику качества записи

# Identify EEG records that have more than 5,4,3 unique expert consensus values

eeg_id_with_multiple_consensus_5 = counts[counts > 5].index.tolist()
eeg_id_with_multiple_consensus_4 = counts[counts > 4].index.tolist()
eeg_id_with_multiple_consensus_3 = counts[counts > 3].index.tolist()

# Print the lists of EEG IDs 

print(eeg_id_with_multiple_consensus_5)
print(eeg_id_with_multiple_consensus_4)
print(eeg_id_with_multiple_consensus_3)

In [ ]:
# Load a specific EEG data file for analysis. We can review any files with 4 or 3 consensus levels.
# Update: Treating category 3 as potentially problematic

data_eeg_path = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1460778765.parquet'

df_multiple = df[df['eeg_id']==1460778765]
data = pd.read_parquet(data_eeg_path).copy()

Настройки каналов

In [ ]:
# Drop the 'EKG' column because it's not present in the montage,
# If not dropped now, all would be dropped later
data = data.drop(['EKG'], axis=1) 
ch_names = data.columns.tolist()

# Create a list of channel types, assuming all channels are EEG
ch_types = ['eeg']*len(ch_names) 

# Create Info object for MNE, setting the sample frequency to 200 Hz
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=200)
print(info)

Creating Raw

In [ ]:
# Convert the DataFrame to a NumPy array and transpose it to fit MNE's data structure requirements
data_values = data.values.T
# Create an MNE RawArray object with the data and info structure
raw = mne.io.RawArray(data_values, info)
# Apply a notch filter at 50 Hz and bandpass filter from 0.1 to 45 Hz to the data
raw = raw.notch_filter(50).filter(0.1, 45)
# Define scaling factors for plotting EEG data
scalings = {'eeg': 300} 
# Calculate the maximum duration to plot based on EEG label offsets, adding 50 seconds margin
duration = df_multiple['eeg_label_offset_seconds'].max()+50
# Set the standard 10-20 montage for EEG
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
# Plot the sensor positions with channel names
raw.set_montage(ten_twenty_montage)
# Plot the raw EEG data without scrollbars and scale bars, using the defined duration and scalings
raw.plot_sensors(show_names=True)
plt.tight_layout()

# график для простмотра и отчистки данных
raw.plot(show_scrollbars=False, show_scalebars=False, duration= duration, scalings=scalings)
plt.tight_layout()

Events and epochs

In [ ]:
# Define a mapping from event types to integers
Target = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
event_ids = {'Seizure':0, 'LPD':1, 'LRDA':3, 'GRDA':4, 'Other':5}
# Prepare events data by mapping expert_consensus using event_ids and adjusting time offsets
events = df_multiple[['eeg_label_offset_seconds', 'expert_consensus']]
events.insert(1, 'New', 0)
events.loc[:,'expert_consensus'] = events['expert_consensus'].map(event_ids)
events.loc[:,'eeg_label_offset_seconds'] = (events['eeg_label_offset_seconds']+25)*200
events = events.values.astype(int)

In [ ]:
# Backup of old events for comparison
events_old = events 
# Plot the events to visualize their distribution in the data
mne.viz.plot_events(events[:])
plt.tight_layout()

In [ ]:
# Create Epochs from the raw data using the event markers, with each epoch from -5 to +5 seconds around the event
epochs = mne.Epochs(raw, events, event_id = event_ids, tmin=-5, tmax=5, preload=True, baseline=(None, 0))
# Plot the first 10 epochs with defined settings, showing all picks without scrollbars or scale bars
epochs.plot(n_epochs=10, events=True, picks = 'all', show_scrollbars=False, show_scalebars=False, scalings=scalings)
plt.tight_layout() # Fix overlapping epochs if necessary

Epochs fix

In [ ]:
# Select 'eeg_label_offset_seconds' and 'expert_consensus' columns from df_multiple
events = df_multiple[['eeg_label_offset_seconds', 'expert_consensus']]
# Initialize a DataFrame to store non-overlapping events    
non_overlapping_events = pd.DataFrame(columns=events.columns)

# Convert 'eeg_label_offset_seconds' column to a list for processing
list_eeg_label_offset_seconds = list(df_multiple['eeg_label_offset_seconds'])
new_list = [] # List to store the new, non-overlapping event offsets
current_offset = 0# Start with the initial offset
min_distance = 10 # Minimum distance between events

# Process to filter out overlapping events based on min_distance
while current_offset <= max(list_eeg_label_offset_seconds):
    new_list.append(current_offset)
    # Find the next event that is at least min_distance away
    next_offset = next((x for x in list_eeg_label_offset_seconds if x >= current_offset + min_distance), None)
    if next_offset is None:
        break
    current_offset = next_offset

# Select events that are in the new_list of non-overlapping events
events = df_multiple[['eeg_label_offset_seconds', 'expert_consensus']]
non_overlapping_events = pd.DataFrame(columns=events.columns)

mask = events['eeg_label_offset_seconds'].isin(new_list)
non_overlapping_events = events[mask]
# Insert a new column with default value 0
non_overlapping_events.insert(1, 'New', 0)
# Map 'expert_consensus' to the corresponding numeric event ID
non_overlapping_events.loc[:,'expert_consensus'] = non_overlapping_events['expert_consensus'].map(event_ids)
# Adjust the 'eeg_label_offset_seconds' and scale by the sampling frequency
non_overlapping_events.loc[:,'eeg_label_offset_seconds'] = (non_overlapping_events['eeg_label_offset_seconds']+25)*200
non_overlapping_events = non_overlapping_events.values.astype(int)

In [ ]:
# Plot original events for comparison
mne.viz.plot_events(events_old[:])
plt.tight_layout()

# Plot non-overlapping events to show the filtering effect
mne.viz.plot_events(non_overlapping_events[:])
plt.tight_layout()

In [ ]:
# Create Epochs from the raw data using non-overlapping events
epochs = mne.Epochs(raw, non_overlapping_events, event_id = event_ids, tmin=-5, tmax=5, preload=True, baseline=(None, 0))
# Plot the first 10 epochs, showing the effect of non-overlapping event selection
epochs.plot(n_epochs=10, events=True, picks = 'all', show_scrollbars=False, show_scalebars=False, scalings=scalings)
plt.tight_layout()

PSD

In [ ]:
# Plot power spectral density for 'Seizure' events between 0.01 and 20 Hz
epochs['Seizure'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
# Plot power spectral density for 'LPD' events between 0.01 and 20 Hz
epochs['LPD'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
# Plot power spectral density for 'LRDA' events between 0.01 and 20 Hz
epochs['LRDA'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
# Plot power spectral density for 'GRDA' events between 0.01 and 20 Hz
epochs['GRDA'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

In [ ]:
# Plot power spectral density for 'Other' events between 0.01 and 20 Hz
epochs['Other'].plot_psd(fmin=0.01, fmax=20)
plt.tight_layout()

Time frequency analysis

In [ ]:
# Perform time-frequency analysis using the Morlet wavelet transform for 'Seizure' events

from mne.time_frequency import tfr_morlet, tfr_multitaper, tfr_stockwell
freq = np.arange(0.5, 20, 0.01) # Define the range of frequencies
n_cycles = freq/2 # Define the number of cycles for the wavelet transform

# Compute the power for 'Seizure' events
power_seizure = tfr_morlet(epochs['Seizure'][0], freq, n_cycles = n_cycles, return_itc = False)
# Plot the time-frequency representation for each channel
for title in ch_names:
    power_seizure.plot(picks=title, title=title)
    plt.tight_layout()

Evoked

In [ ]:
# Compute the average of the epochs to get the evoked response. 
# This collapses the data across epochs to get the mean evoked potential for each channel.

evoked = epochs.average() 
evoked

In [ ]:
# Display the mapping of event IDs used in the epochs object.

epochs.event_id

In [ ]:
# Initialize a dictionary to hold the averaged evoked data for each condition.
my_evokeds = {}

# Loop over each condition in the event ID dictionary.
for condition in epochs.event_id:
    # Loop over each condition in the event ID dictionary.
    my_evokeds[condition] = epochs[condition].average()

# Print the dictionary of evoked responses.
my_evokeds

In [ ]:
# Loop over specified conditions and compute the mean amplitude in a specific time window.

for condition in ['Seizure', 'LPD', 'LRDA', 'GRDA', 'Other']:
    # Copy the evoked data for the condition, pick the 'Cz' channel, and find the peak in the specified time range.
    chan, lat, amp = my_evokeds[condition].copy().pick_channels(['Cz']).get_peak(tmin = -2, tmax = 0, mode = 'neg', return_amplitude = True)
    print(condition)
    # Print the latency and amplitude (converted to microvolts) of the peak.
    print(lat, amp * 1e6)

In [ ]:
for condition in ['Seizure', 'LPD', 'LRDA', 'GRDA', 'Other']:
    # Copy the evoked data for the condition, pick the 'Cz' channel, crop the data to the time window, and calculate the mean.
    amp = my_evokeds[condition].copy().pick_channels(['Cz']).crop(tmin = -2, tmax = 0).data.squeeze().mean()
   # print(vector.shape)
    print(condition)
    # Print the mean amplitude (converted to microvolts).
    print(amp * 1e6)

ERPs

In [ ]:
# Plot the evoked responses for all conditions on the same plot for comparison.
# The plot is inverted in the y-axis to match the EEG/MEG convention in some publications.

mne.viz.plot_compare_evokeds(my_evokeds, picks = ['Cz'], invert_y = True) # как то грязно 

In [ ]:
# Filter the evoked responses to only include the 'Cz' channel.
my_evokeds = {condition: evoked.copy().pick_channels(['Cz']) for condition, evoked in my_evokeds.items()}

In [ ]:
# Print the channel names for the 'Seizure' condition to verify the channel picking.
my_evokeds['Seizure'].ch_names

In [ ]:
# Plot the evoked response for the 'Seizure' condition.
mne.viz.plot_compare_evokeds(my_evokeds['Seizure'], picks = ['Cz'], invert_y = True) 

In [ ]:
# Plot the evoked response for the 'LPD' condition with a specific color.
mne.viz.plot_compare_evokeds(my_evokeds['LPD'], picks = ['Cz'], invert_y = True, colors = ['orange']) 

In [ ]:
# Plot the evoked response for the 'LRDA' condition with a specific color.
mne.viz.plot_compare_evokeds(my_evokeds['LRDA'], picks = ['Cz'], invert_y = True, colors = ['green']) 

In [ ]:
# Plot the evoked response for the 'GRDA' condition with a specific color.
mne.viz.plot_compare_evokeds(my_evokeds['GRDA'], picks = ['Cz'], invert_y = True, colors = ['red']) 

In [ ]:
# Plot the evoked response for the 'Other' condition with a specific color.
mne.viz.plot_compare_evokeds(my_evokeds['Other'], picks = ['Cz'], invert_y = True,  colors = ['purple']) 